# VITTA: Visibilidade para salvar vidas

## Objetivo do Projeto:

- Desenvolver uma ferramenta de apoio à logística de unidades hospitalares, com foco na visualização de estoques, previsão de consumo e roteirização de abastecimento. Este sistema visa antecipar riscos de desabastecimento e facilitar a tomada de decisão na gestão de materiais.

- Integrantes:

    Djalma Andrade - RM:555530

    Felipe Carioba - RM:558447

    Caio Felipe de Lima Bezerra - RM: 556197


## Considerações do Projeto
- Simula uma rede de unidades de saúde com localizações reais em São Paulo e Campinas.

- Trabalha com materiais críticos, como EPIs, medicamentos e insumos médicos.

- Utiliza conceitos de grafos para roteirização e algoritmos BFS/DFS para análise da rede.

- Gera previsões de consumo com memoização.

- Visualiza os dados em mapas interativos.

## Preparação do Ambiente
- Instalação de bibliotecas necessárias para mapas

In [1]:
!pip install folium


## Importação das bibliotecas e Definição de Funções Auxiliares

- Decorator de Memoização
- Decorator para medição de tempo
- Decorator para exibição de logs

In [2]:
import folium
import logging
import time
import datetime
import tracemalloc
import random
from datetime import date, timedelta
from collections import deque
from IPython.display import display

logging.basicConfig(level=logging.INFO)

def logger(func):
    def wrapper(*args, **kwargs):
        logging.info(f"Executando {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

# Lista global para armazenar resultados de desempenho
resultados = []

def performance(func):
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        start_time = time.time()
        result = func(*args, **kwargs)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        tracemalloc.stop()

        tempo = 1000 * (end_time - start_time)
        memoria = peak / 1024

        resultados.append((func.__name__, tempo, memoria))
        print(f"{func.__name__} -> Tempo: {tempo:.2f} ms | Memória: {memoria:.2f} KB")

        return result
    return wrapper

def memoize(func):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrapper


## Roteirização Logística - Modelagem com Grafos
- Definição das Unidades
- Nomes dos Materiais Controlados
- Definição do Grafo de Logística

In [3]:
unidades = [
    {"nome": "Hospital Central", "lat": -23.5505, "lon": -46.6333},
    {"nome": "Unidade Leste", "lat": -23.5641, "lon": -46.4742},
    {"nome": "Unidade Oeste", "lat": -23.5562, "lon": -46.6784},
    {"nome": "Unidade Norte", "lat": -23.4844, "lon": -46.6206},
    {"nome": "Unidade Sul", "lat": -23.6512, "lon": -46.7073},
    {"nome": "Unidade Campinas", "lat": -22.9099, "lon": -47.0626}
]

materiais = ["Luvas", "Máscaras", "Seringas", "Medicamento A", "Medicamento B", "EPIs"]

estoque = {
    "Hospital Central": {"Luvas": 50, "Máscaras": 30, "Seringas": 25, "Medicamento A": 10, "Medicamento B": 15, "EPIs": 20},
    "Unidade Leste":    {"Luvas": 20, "Máscaras": 15, "Seringas": 10, "Medicamento A": 5,  "Medicamento B": 8,  "EPIs": 12},
    "Unidade Oeste":    {"Luvas": 80, "Máscaras": 40, "Seringas": 35, "Medicamento A": 20, "Medicamento B": 25, "EPIs": 30},
    "Unidade Norte":    {"Luvas": 10, "Máscaras": 5,  "Seringas": 8,  "Medicamento A": 3,  "Medicamento B": 4,  "EPIs": 5},
    "Unidade Sul":      {"Luvas": 25, "Máscaras": 18, "Seringas": 12, "Medicamento A": 7,  "Medicamento B": 10, "EPIs": 14},
    "Unidade Campinas": {"Luvas": 100,"Máscaras": 60, "Seringas": 50, "Medicamento A": 30, "Medicamento B": 40, "EPIs": 50}
}

estoque_ideal = {
    "Luvas": 40,
    "Máscaras": 25,
    "Seringas": 20,
    "Medicamento A": 10,
    "Medicamento B": 15,
    "EPIs": 20
}

grafo = {
    "Hospital Central": ["Unidade Leste", "Unidade Oeste"],
    "Unidade Leste": ["Unidade Norte"],
    "Unidade Oeste": ["Unidade Sul"],
    "Unidade Norte": ["Unidade Campinas"],
    "Unidade Sul": [],
    "Unidade Campinas": []
}


## Algoritmos de Percurso

- BFS - Busca em Largura
- DFS - Busca em Profundidade

In [4]:
def bfs(origem):
    visitados = set()
    fila = deque([origem])
    resultado = []
    while fila:
        atual = fila.popleft()
        if atual not in visitados:
            resultado.append(atual)
            visitados.add(atual)
            fila.extend(grafo.get(atual, []))
    return resultado

def dfs(origem, visitados=None):
    if visitados is None:
        visitados = set()
    visitados.add(origem)
    for vizinho in grafo.get(origem, []):
        if vizinho not in visitados:
            dfs(vizinho, visitados)
    return visitados

## Gestão de Estoque e Consumo
- Árvore de Consumo
- Inserção dos Dados de Consumo dos Materiais
- Listagem dos Materiais por Ordem de Consumo
- Previsão de Consumo por Unidade
- Priorização de Reabastecimento


In [5]:
arvore_consumo = []

def inserir_consumo(material, consumo):
    arvore_consumo.append((material, consumo))

for i in range(len(materiais)):
    inserir_consumo(materiais[i], sum(estoque[u["nome"]][materiais[i]] for u in unidades) // len(unidades))

def listar_em_ordem(lista):
    return sorted(lista, key=lambda x: x[1])

@memoize
def prever_consumo(dias, consumo_diario):
    if dias == 0:
        return 0
    return consumo_diario + prever_consumo(dias - 1, consumo_diario)

@logger
@performance
def produtos_em_falta():
    resultado = {}
    for unidade in estoque:
        resultado[unidade] = {}
        for item, quantidade in estoque[unidade].items():
            if quantidade < estoque_ideal[item]:
                resultado[unidade][item] = quantidade
        if not resultado[unidade]:
            resultado.pop(unidade)
    return resultado

@logger
@performance
def produtos_sobrando():
    resultado = {}
    for unidade in estoque:
        resultado[unidade] = {}
        for item, quantidade in estoque[unidade].items():
            if quantidade > estoque_ideal[item]:
                resultado[unidade][item] = quantidade
        if not resultado[unidade]:
            resultado.pop(unidade)
    return resultado

@performance
def reabastecimento_prioritario(item):
    ranking = sorted(estoque.items(), key=lambda x: x[1][item])
    return ranking


## Registro diário (Fila) e Pilha de consultas

A seguir implementamos:
- **Fila (`registro_diario`)**: armazena o consumo diário de insumos em ordem cronológica. Cada entrada contém a data e um dicionário `material -> quantidade` para aquele dia.

- **Pilha (`pilha_consultas`)**: armazena consultas/ações em LIFO (últimas ações/consultas primeiro) — útil para auditoria rápida e desfazer operações.

As entradas de consumo atualizam **`arvore_consumo`** (agregado de consumo por material), mantendo consistência.

In [6]:
registro_diario = deque()

pilha_consultas = []

def registrar_consumo_do_dia(data, consumo_por_material, atualizar_estoque=False):
    registro_diario.append({"data": data, "consumo": consumo_por_material})

    for m, qtd in consumo_por_material.items():
        for idx, (mat, tot) in enumerate(arvore_consumo):
            if mat == m:
                arvore_consumo[idx] = (mat, tot + qtd)
                break
        else:
            arvore_consumo.append((m, qtd))

    if atualizar_estoque:
        unidades_nomes = list(estoque.keys())
        n_unidades = len(unidades_nomes) if unidades_nomes else 1
        for m, qtd in consumo_por_material.items():
            share = qtd // n_unidades
            for un in unidades_nomes:
                estoque[un][m] = max(0, estoque[un].get(m, 0) - share)

    return True

def consultar_ultimos_dias(n=5):
    resultado = list(registro_diario)[-n:]
    pilha_consultas.append(("consulta_ultimos_dias", n))
    return resultado

hoje = date.today()
for dias_atras in range(7):
    data_d = hoje - timedelta(days=6 - dias_atras)
    consumo_do_dia = {m: random.randint(0, 12) for m in materiais}
    registrar_consumo_do_dia(data_d, consumo_do_dia, atualizar_estoque=False)


## Buscas no registro de consumo

Agora implementamos:
- **Busca sequencial** no `registro_diario` (varredura por dia) para localizar em quais dias e quanto um insumo foi consumido.

- **Busca binária** aplicada a uma *lista agregada* (`aggregated_consumo`) ordenada por nome do insumo — mais eficiente para consultas por nome quando a lista está ordenada.

As buscas retornam detalhes por dia e um total agregado, úteis para auditoria/relatórios.


In [7]:
def aggregate_consumo_from_registro():
    agg = {}
    for rec in registro_diario:
        for m, q in rec["consumo"].items():
            agg[m] = agg.get(m, 0) + q
    return [(m, agg[m]) for m in agg]

def busca_sequencial_registro(material):
    detalhes = []
    total = 0
    for rec in registro_diario:
        q = rec["consumo"].get(material, 0)
        if q:
            detalhes.append((rec["data"], q))
            total += q
    if not detalhes:
        return -1, None
    return detalhes, total

def busca_binaria_agrupada(material):
    agg = sorted(aggregate_consumo_from_registro(), key=lambda x: x[0])
    inicio, fim = 0, len(agg) - 1
    while inicio <= fim:
        meio = (inicio + fim) // 2
        if agg[meio][0] == material:
            return meio, agg[meio]
        elif agg[meio][0] < material:
            inicio = meio + 1
        else:
            fim = meio - 1
    return -1, None


## Ordenação de insumos por consumo ou validade

Implementamos versões parametrizáveis de **Merge Sort** e **Quick Sort** que aceitam uma função `key`.
Usaremos:
- ordenação por **quantidade consumida** (a partir da agregação do registro diário),

- e ordenação por **validade** (simulada aqui via metadados `materiais_info`) — útil para priorizar uso/transferência de lotes que vencem antes.


In [8]:
def merge_sort_custom(lista, key=lambda x: x[1]):
    if len(lista) <= 1:
        return lista
    meio = len(lista) // 2
    esquerda = merge_sort_custom(lista[:meio], key)
    direita = merge_sort_custom(lista[meio:], key)
    return merge_custom(esquerda, direita, key)

def merge_custom(esq, dir_, key):
    res = []
    i = j = 0
    while i < len(esq) and j < len(dir_):
        if key(esq[i]) <= key(dir_[j]):
            res.append(esq[i]); i += 1
        else:
            res.append(dir_[j]); j += 1
    res.extend(esq[i:]); res.extend(dir_[j:])
    return res

def quick_sort_custom(lista, key=lambda x: x[1]):
    if len(lista) <= 1:
        return lista
    pivo = lista[len(lista)//2]
    menores = [x for x in lista if key(x) < key(pivo)]
    iguais = [x for x in lista if key(x) == key(pivo)]
    maiores = [x for x in lista if key(x) > key(pivo)]
    return quick_sort_custom(menores, key) + iguais + quick_sort_custom(maiores, key)

materiais_info = {}
base = datetime.datetime.now()
for i, m in enumerate(materiais):
    materiais_info[m] = {"validade": base + timedelta(days=30*(i+1))}

agg = aggregate_consumo_from_registro()

ordenado_por_qtd_merge = merge_sort_custom(agg, key=lambda x: x[1])
ordenado_por_qtd_quick = quick_sort_custom(agg, key=lambda x: x[1])

agg_com_validade = [(m, qtd, materiais_info.get(m, {}).get("validade")) for m, qtd in agg]
ordenado_por_validade = quick_sort_custom(agg_com_validade, key=lambda x: x[2])

## Integração: usar Fila + Buscas + Ordenação no fluxo de reabastecimento

Demonstração prática:
1. Agregar consumo por material a partir do `registro_diario`.

2. Estimar taxa diária média e previsão (usando `prever_consumo`) para os próximos X dias.

3. Enfileirar (FIFO) pedidos de reabastecimento para unidades cujo estoque cai abaixo da previsão; registrar as ações na pilha (LIFO) para auditoria/desfazer.


In [9]:
fila_reabastecimento = deque()

agg_dict = dict(aggregate_consumo_from_registro())
dias_registrados = len(registro_diario) if len(registro_diario) > 0 else 1

for unidade_nome, itens in estoque.items():
    for item in materiais:
        taxa_diaria = agg_dict.get(item, 0) / dias_registrados
        previsao_7dias = prever_consumo(7, taxa_diaria)
        limite = previsao_7dias + 0.10 * estoque_ideal[item]
        if estoque[unidade_nome][item] < limite:
            qtd_sugerida = max(int(previsao_7dias), int(estoque_ideal[item] - estoque[unidade_nome][item]))
            fila_reabastecimento.append((unidade_nome, item, qtd_sugerida))

## Visualização no Mapa das Unidades  

- **Criação do Mapa Dinâmico**  
  - Geração de um mapa interativo usando **Folium**, centralizado em São Paulo.  

- **Exibição das Unidades de Saúde**  
  - Cada unidade aparece como um marcador no mapa.  
  - A cor do marcador indica a situação do estoque:  
    - 🟢 **Verde** → Estoque normal.  
    - 🔴 **Vermelho** → Estoque crítico (detecção feita por `produtos_em_falta`).  

- **Informações no Popup**  
  - Nome da unidade.  
  - Estoque total consolidado.  
  - Lista de **produtos críticos** em falta (quando houver).  
  - **Previsão de consumo em 7 dias** para cada material, calculada a partir da média de consumo registrada.  

- **Rede de Conexões (Grafo)**  
  - Linhas azuis representam as **arestas** da rede entre as unidades.  
  - Mostra como os hospitais/unidades estão interligados para suporte logístico.  

- **Rotas de Roteirização**  
  - **BFS (Busca em Largura)** → rota otimizada para cobrir as unidades em menor número de passos, destacada em **laranja**.  
  - **DFS (Busca em Profundidade)** → rota alternativa que percorre unidades em profundidade, destacada em **roxo**.  



In [10]:
mapa = folium.Map(
    location=[-23.55, -46.63],
    zoom_start=11,
)

for u in unidades:
    nome = u["nome"]
    total_estoque = sum(estoque[nome].values())

    falta_unidade = produtos_em_falta().get(nome, {})
    cor = "red" if falta_unidade else "green"

    previsoes = []
    for item in materiais:
        taxa_diaria = agg_dict.get(item, 0) / dias_registrados
        previsao = prever_consumo(7, taxa_diaria)
        previsoes.append(f"{item}: {previsao:.1f}")

    popup_text = (
        f"<b>{nome}</b><br>"
        f"Estoque Total: {total_estoque}<br>"
        f"<b>Produtos críticos:</b> {', '.join(falta_unidade.keys()) if falta_unidade else 'Nenhum'}<br>"
        f"<b>Previsão 7 dias:</b><br>" + "<br>".join(previsoes)
    )

    folium.Marker(
        location=[u["lat"], u["lon"]],
        popup=popup_text,
        icon=folium.Icon(color=cor)
    ).add_to(mapa)

for origem, destinos in grafo.items():
    coord_origem = next(( (u["lat"], u["lon"]) for u in unidades if u["nome"] == origem ), None)
    for destino in destinos:
        coord_destino = next(( (u["lat"], u["lon"]) for u in unidades if u["nome"] == destino ), None)
        if coord_origem and coord_destino:
            folium.PolyLine(
                [coord_origem, coord_destino],
                color="blue",
                weight=2,
                opacity=0.6
            ).add_to(mapa)

rota_bfs = bfs("Hospital Central")
for i in range(len(rota_bfs) - 1):
    origem = next(( (u["lat"], u["lon"]) for u in unidades if u["nome"] == rota_bfs[i] ), None)
    destino = next(( (u["lat"], u["lon"]) for u in unidades if u["nome"] == rota_bfs[i+1] ), None)
    if origem and destino:
        folium.PolyLine(
            [origem, destino],
            color="orange",
            weight=4,
            opacity=0.9
        ).add_to(mapa)

rota_dfs = list(dfs("Hospital Central"))
rota_dfs = list(rota_dfs)
for i in range(len(rota_dfs) - 1):
    origem = next(( (u["lat"], u["lon"]) for u in unidades if u["nome"] == rota_dfs[i] ), None)
    destino = next(( (u["lat"], u["lon"]) for u in unidades if u["nome"] == rota_dfs[i+1] ), None)
    if origem and destino:
        folium.PolyLine(
            [origem, destino],
            color="purple",
            weight=3,
            opacity=0.7
        ).add_to(mapa)

display(mapa)



produtos_em_falta -> Tempo: 0.06 ms | Memória: 1.02 KB
produtos_em_falta -> Tempo: 0.03 ms | Memória: 1.02 KB
produtos_em_falta -> Tempo: 0.03 ms | Memória: 1.02 KB
produtos_em_falta -> Tempo: 0.02 ms | Memória: 1.02 KB
produtos_em_falta -> Tempo: 0.02 ms | Memória: 1.14 KB
produtos_em_falta -> Tempo: 0.02 ms | Memória: 1.20 KB


## Análise dos Dados Consolidada  

- **Unidades com Estoque Crítico**  
  - Identificação de quais unidades apresentam insumos abaixo do nível mínimo.  
  - Lista dos produtos críticos por unidade.  

- **Previsão de Consumo em 7 Dias**  
  - Estimativa baseada na taxa média de consumo diário.  
  - Mostra o consumo esperado para cada material no período.  

- **Roteirização da Rede (BFS e DFS)**  
  - **BFS (Busca em Largura):** encontra o caminho mais curto para visitar as unidades.  
  - **DFS (Busca em Profundidade):** percorre a rede em profundidade para análise alternativa de rota.  

- **Materiais Ordenados por Consumo Médio**  
  - Organização dos insumos com base no consumo médio.  
  - Aplicação de **algoritmos de ordenação** (Merge Sort e Quick Sort).  

- **Estruturas de Dados Aplicadas**  
  - **Fila (Queue):** registra o consumo em ordem cronológica (primeiro a entrar, primeiro a sair).  
  - **Pilha (Stack):** permite consultar os últimos consumos primeiro (último a entrar, primeiro a sair).  

- **Buscas em Estruturas**  
  - **Busca Sequencial:** percorre todos os registros até encontrar o insumo desejado.  
  - **Busca Binária:** pesquisa otimizada em registros previamente ordenados.  

- **Organização de Insumos**  
  - Ordenação por quantidade consumida.  
  - Ordenação por validade dos insumos.  


In [11]:
print("\n Unidades com Estoque Crítico:")
print(produtos_em_falta())

print("\n Previsão de Consumo em 7 dias por material:")
agg_dict = dict(aggregate_consumo_from_registro())
dias_registrados = len(registro_diario) if len(registro_diario) > 0 else 1
for item in materiais:
    taxa_diaria = agg_dict.get(item, 0) / dias_registrados
    previsao = prever_consumo(7, taxa_diaria)
    print(f"- {item}: {previsao:.2f} unidades previstas")

print("\n Roteirização da Rede (BFS):")
print(bfs("Hospital Central"))

print("\n Roteirização da Rede (DFS):")
print(dfs("Hospital Central"))

print("\n Materiais Ordenados por Consumo Médio:")
for mat, qtd in listar_em_ordem(arvore_consumo):
    print(f"- {mat}: {qtd}")

print("\n Registro de Consumo Diário (Fila - ordem cronológica):")
for r in registro_diario:
    print(r)

print("\n Consultas em Ordem Inversa (Pilha - últimos consumos primeiro):")
print(list(reversed(pilha_consultas)))

print("\n Busca Sequencial no Registro (exemplo: 'Luvas'):")
print(busca_sequencial_registro("Luvas"))

print("\n Busca Binária no Registro (exemplo: 'Luvas'):")
print(busca_binaria_agrupada("Luvas"))

print("\n Ordenação dos Insumos por Quantidade Consumida (Merge Sort):")
print(ordenado_por_qtd_merge)

print("\n Ordenação dos Insumos por Quantidade Consumida (Quick Sort):")
print(ordenado_por_qtd_quick)

print("\n Ordenação dos Insumos por Validade (Quick Sort):")
print(ordenado_por_validade)



 Unidades com Estoque Crítico:
produtos_em_falta -> Tempo: 0.06 ms | Memória: 0.99 KB
{'Unidade Leste': {'Luvas': 20, 'Máscaras': 15, 'Seringas': 10, 'Medicamento A': 5, 'Medicamento B': 8, 'EPIs': 12}, 'Unidade Norte': {'Luvas': 10, 'Máscaras': 5, 'Seringas': 8, 'Medicamento A': 3, 'Medicamento B': 4, 'EPIs': 5}, 'Unidade Sul': {'Luvas': 25, 'Máscaras': 18, 'Seringas': 12, 'Medicamento A': 7, 'Medicamento B': 10, 'EPIs': 14}}

 Previsão de Consumo em 7 dias por material:
- Luvas: 39.00 unidades previstas
- Máscaras: 37.00 unidades previstas
- Seringas: 39.00 unidades previstas
- Medicamento A: 43.00 unidades previstas
- Medicamento B: 56.00 unidades previstas
- EPIs: 39.00 unidades previstas

 Roteirização da Rede (BFS):
['Hospital Central', 'Unidade Leste', 'Unidade Oeste', 'Unidade Norte', 'Unidade Sul', 'Unidade Campinas']

 Roteirização da Rede (DFS):
{'Unidade Campinas', 'Hospital Central', 'Unidade Norte', 'Unidade Leste', 'Unidade Oeste', 'Unidade Sul'}

 Materiais Ordenados p

## Conclusão
O sistema desenvolvido permite:

- Visualizar as unidades de saúde em um mapa interativo, identificando facilmente os estoques críticos.

- Planejar a roteirização da logística de abastecimento utilizando algoritmos de grafos (BFS e DFS).

- Prever consumo futuro com base no consumo médio diário, auxiliando na tomada de decisão.

- Organizar e priorizar o envio de materiais conforme a necessidade de cada unidade.

## Melhorias Futuras
- Integração com APIs de rotas (como Google Maps ou OpenRouteService) para otimizar trajetos.

- Gestão dinâmica de múltiplos materiais e múltiplos tipos de consumo.

- Interface gráfica para usuários não técnicos.

- Simulação de entrega, considerando autonomia, carga e pontos de recarga.